In [1]:
import sys
import os 
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup 
import requests
import lxml
import json

project_path = Path(os.path.dirname(os.path.realpath("__file__"))).parent

from scripts.utils import log_df

In [2]:
constellation_path = project_path/'raw_data'/'constellations.csv'
constellation_js_path = project_path/'data'/'constellations.js'

## get constellation data

In [3]:
wikiurl="https://en.wikipedia.org/wiki/IAU_designated_constellations"
# table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [4]:
soup = BeautifulSoup(response.text, 'html.parser')
table=soup.find('table',{'class':"wikitable"})

In [5]:
df = pd.read_html(str(table))
df = pd.DataFrame(df[0])
df.columns = ['Constellation', 'Abbreviations IAU',
              'Abbreviations NASA', 'Genitive',
              'Origin', 'Meaning', 'Brightest star']
log_df(df)

(88, 7)


,Constellation,Abbreviations IAU,Abbreviations NASA,Genitive,Origin,Meaning,Brightest star
0,Andromeda /ænˈdrɒmɪdə/[7],And,Andr,Andromedae /ænˈdrɒmɪdiː/,ancient (Ptolemy),Andromeda (The chained maiden or princess),Alpheratz
1,Antlia /ˈæntliə/[7],Ant,Antl,Antliae /ˈæntliiː/,"1763, Lacaille",air pump,α Antliae
2,Apus /ˈeɪpəs/[8],Aps,Apus,Apodis /ˈæpoʊdɪs/[8],"1603, Uranometria, created by Keyser and de Ho...",Bird-of-paradise/Exotic Bird/Extraordinary Bird,α Apodis
3,Aquarius /əˈkwɛəriəs/[7],Aqr,Aqar,Aquarii /əˈkwɛəriaɪ/,ancient (Ptolemy),water-bearer,Sadalsuud
4,Aquila /ˈækwɪlə/[7],Aql,Aqil,Aquilae /ˈækwɪliː/,ancient (Ptolemy),eagle,Altair


In [6]:
df[['name', 'b']] = df['Constellation'].str.split(' /', expand=True)
del df['b']
log_df(df)

(88, 8)


,Constellation,Abbreviations IAU,Abbreviations NASA,Genitive,Origin,Meaning,Brightest star,name
0,Andromeda /ænˈdrɒmɪdə/[7],And,Andr,Andromedae /ænˈdrɒmɪdiː/,ancient (Ptolemy),Andromeda (The chained maiden or princess),Alpheratz,Andromeda
1,Antlia /ˈæntliə/[7],Ant,Antl,Antliae /ˈæntliiː/,"1763, Lacaille",air pump,α Antliae,Antlia
2,Apus /ˈeɪpəs/[8],Aps,Apus,Apodis /ˈæpoʊdɪs/[8],"1603, Uranometria, created by Keyser and de Ho...",Bird-of-paradise/Exotic Bird/Extraordinary Bird,α Apodis,Apus
3,Aquarius /əˈkwɛəriəs/[7],Aqr,Aqar,Aquarii /əˈkwɛəriaɪ/,ancient (Ptolemy),water-bearer,Sadalsuud,Aquarius
4,Aquila /ˈækwɪlə/[7],Aql,Aqil,Aquilae /ˈækwɪliː/,ancient (Ptolemy),eagle,Altair,Aquila


In [7]:
 df.to_csv(constellation_path, index=False)

In [8]:
object_dict = {}
for index, row in df.iterrows():
    object_dict[row['Abbreviations IAU']] = row['name']

json_object = json.dumps(object_dict)
 
with open(constellation_js_path, "w") as outfile:
    outfile.write(json_object)